In [ ]:
from nltk.corpus import wordnet as wn
import json
from itertools import chain
import json
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('wordnet2022')
nltk.download('words')
!pip install language-tool-python
import language_tool_python
import re
import spacy
nlp = spacy.load("en_core_web_sm")
!pip install --upgrade gingerit
import gingerit




! cp -rf /usr/share/nltk_data/corpora/wordnet2022 /usr/share/nltk_data/corpora/wordnet # temp fix for lookup error

In [ ]:
def is_proper_english(sentence):
    # Download the necessary resources for the NLTK library
    
    
    # Tokenize the sentence into individual words
    words = nltk.word_tokenize(sentence)
    
    # Tag each word with its part of speech
    tagged_words = nltk.pos_tag(words)
    
    # Check if all words are recognized as English words and their corresponding part of speech is valid
    for word, tag in tagged_words:
        if not word.isalpha() or tag not in ['JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS', 'RB', 'RBR', 'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
            return False
    
    return True

# def check_english_sentence(sentence):
#     sentence = re.sub(r'[^\w\s]', ' ', sentence)

#     # Remove extra whitespace and convert to lowercase
#     sentence = re.sub(r'\s+', ' ', sentence).strip().lower()
#     words = nltk.word_tokenize(sentence)
#     print(words)
#     english_vocab = set(w.lower() for w in nltk.corpus.words.words())
#     return all(word.lower() in english_vocab for word in words)

def check_english_sentence(sentence):
    sentence = re.sub(r'[^\w\s]', ' ', sentence)
    sentence = re.sub(r'\s+', ' ', sentence).strip().lower()
    doc = nlp(sentence)

    return all(token.is_alpha and token.is_ascii and token.lang_ == "en" for token in doc)
def remove_symbols(sentence):
    # remove mathematical symbols
    sentence = re.sub(r'[+\-*\/=<>]', '', sentence)

    # remove currency symbols
#     sentence = re.sub(r'[£€¥$]', '', sentence)

    # remove special characters except for parentheses, semicolons, exclamation marks, and quotation marks
    sentence = re.sub(r'[^\w\s();!"\']', '', sentence)

    return sentence

def correct_sentence(sentence):
    tool = language_tool_python.LanguageTool('en-US')
    corrected = tool.correct(sentence)
    return corrected

sentence ="Obsolete spelling of bough"
print(check_english_sentence(sentence))
print(remove_symbols(sentence))

In [ ]:
lst = []
with open("/kaggle/input/wiktionarywebster/kaikki.org-dictionary-English.json", "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        lst.append(data)

In [ ]:
word_dict = {}

for json_obj in lst:
    word = json_obj['word']
    if word not in word_dict:
        word_dict[word] = 1
    else:
        word_dict[word] += 1


In [ ]:
def formatUsage(body):
    usageList = []
    definitions = []
    try:
        
        for i in body['senses']:
            definition = i['glosses'][0].split('\n') if 'glosses' in i else [""]
            definition = definition[0]
            if definition in definitions:
                continue
            definitions.append(definition)

            if 'examples' in i:
                for j in i['examples']:
                    j['source'] = 'Wiktionary'
                    j['text'] = remove_symbols(j['text'])

            examples = i['examples'] if 'examples' in i else []
            filtered_data = list(filter(lambda x: check_english_sentence(x['text']) , examples))

            usageList.append({
                'definition': {
                                'gloss':definition,
                                'source':'Wiktionary',
                },
                'examples': filtered_data
            })
    except:
        return usageList
    
    return usageList

def formatAudio(body):
    audioList = []
    if 'sounds' in body:
        for j in body['sounds']:
            if ('mp3_url' in j):
                for i in audioList:
                    if 'tags' in j and  i['tags'] == j['tags'][0]:
                        continue
                audioList.append({
                    'audioLink': j['mp3_url'],
                    'tags' : j['tags'][0] if 'tags' in j else "",
                    'source': 'Wiktionary'

                })
    return audioList            
           




    

def formatObject(body):
    return {
        'pos'  : body['pos'],
        'etymology_text' : body['etymology_text'] if 'etymology_text' in body else "",
        'etymology_number' : body['etymology_number'] if 'etymology_number' in body else "",
        'definitions': formatUsage(body),
        'audio':formatAudio(body),
        'source':'Wiktionary'

    }
    

In [ ]:
payload= {}
count = 0
for i in lst:
    count+=1
    if count % 100000 ==1:
        print(count)
    word = i['word']
    payload[word]= [formatObject(i)] if word not in payload else payload[word]+[formatObject(i)]
  

In [ ]:
print(len(payload))
with open("parsedWiktionary", "w") as f:
    # Write the dictionary to the file in JSON format
    json.dump(payload, f)